In [1]:
!pip3 install pandas
!pip3 install numpy
!pip3 install selenium

In [2]:
import os
import pandas as pd
import time
import numpy as np
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

# Current directory
ROOT_DIR = os.getcwd()

In [3]:
# Load the URL of all kitas
df_kitas_url = pd.read_csv('df_kitas_url.csv')
df_kitas_url.head(3)

,kita_url
0,https://www.kita-navigator.berlin.de/einrichtu...
1,https://www.kita-navigator.berlin.de/einrichtu...
2,https://www.kita-navigator.berlin.de/einrichtu...


In [4]:
# Main Kita webpage
kita_url = 'https://kita-navigator.berlin.de/einrichtungen/363?input=Kolberger%20Str.%2C%2013357%20Berlin%2C%20Alemanha&betb=2-2020&einfacheSuche=false&entfernung=5&lat=52.5455402&lon=13.3773108&seite=1&index=1'

In [5]:
# List that will store temporarily all 
# the information about the kita
kita_info = []

# List that will contain the aufnahmealhalter
# information because it's an array of information
aufnahmealhalter = []

In [6]:
# Chrome driver to open the webpage and get the full html object
webdriver = ROOT_DIR + "/chromedriver"
driver = Chrome(webdriver)

In [7]:
# The driver will open the Chrome 
# page and get the contents
driver.get(kita_url)

# This time will be to wait 2 seconds 
# to give some time for the entire page load
time.sleep(2)

In [8]:
# Right now we'll get the elements 
# inside of the page to extract the infos about the Kita

# Get the Kita address
for elem in driver.find_elements_by_xpath('.//a[@title = "Externer Link Wegbeschreibung in Google Maps (Öffnet neuen Tab)"]'):
    kita_address = elem.text

# Get the Kita website
for elem in driver.find_elements_by_xpath('.//a[@title = "Externer Link Homepage der Kita (Öffnet neuen Tab)"]'):
    kita_website = elem.text
    
# Get the Kita phone
for elem in driver.find_elements_by_xpath('.//a[@title = "Die Kita anrufen"]'):
    kita_telephone = elem.text

# Get the Kita e-mail    
for elem in driver.find_elements_by_xpath('.//a[@title = "Eine E-Mail an die Kita senden"]'):
    kita_email = elem.text

# Get the Kita aufnahmealhalter
for elem in driver.find_elements_by_xpath('.//span[@class = "d-block"]'): 
    aufnahmealhalter.append(str(elem.text).split(' '))
    kita_aufnahmealter = str(elem.text).split(' ')
    
# Using the Kita aufnahmealhalter we will extract more information
kita_distance = ' '.join(aufnahmealhalter[0])
kita_aufnahmealter_jahre = ' '.join(aufnahmealhalter[1])
kita_maximales_platzangebot = ' '.join(aufnahmealhalter[2])
kita_maximales_platzangebot_verfugbar = ' '.join(aufnahmealhalter[3])  


# Store everthing in an array using a tuple
kita_info.append((kita_address,
                  kita_website,
                  kita_telephone,
                  kita_email,
                  kita_aufnahmealter,
                  kita_distance,
                  kita_aufnahmealter_jahre,
                  kita_maximales_platzangebot,
                  kita_maximales_platzangebot_verfugbar                  
                 ))

In [9]:
# Transform in a pandas dataframe just for filtering
df = pd.DataFrame(kita_info)

# Put the right columns
df.columns = ['address',
              'website',
              'phone',
              'email',
              'aufnahmealter',
              'distance',
              'aufnahmealter_jahre',
              'maximales_platzangebot',
              'maximales_platzangebot_verfugbar' 
             ]
# Check
df.head(5)

,address,website,phone,email,aufnahmealter,distance,aufnahmealter_jahre,maximales_platzangebot,maximales_platzangebot_verfugbar
0,"Wiesenstr. 25, 13357 Berlin",http://www.unionhilfswerk.de,+49-30-4622076,kita.wiesenstrasse@unionhilfswerk.de,"[davon, unter, 3, Jahre:, 30]",0.45 km,3 Monate,87,davon unter 3 Jahre: 30


The above code will run OK if there's all information in website (Phone, website, email, etc), it means that **one of those fields is empty, the solution will break**.

In the function below I've included some `try...catch` statements to overcome some of these problems. However the code it'still very fragile yet. 

In [10]:
def get_kita_infos(url_kita):
    try:
        # List that will store temporarily all the information about the kita
        kita_info = []

        # List that will contain the aufnahmealhalter information because it's an array of information
        aufnahmealhalter = []

        # Chrome driver to open the webpage and get the full html object
        webdriver = "/Users/flavio.desouza/Desktop/ml-borracharia/src/kitas-scrapper/chromedriver"
        driver = Chrome(webdriver)

        # The driver will open the Chrome page and get the contents
        driver.get(url_kita)

        # This time will be to wait 2 seconds to give some time for the entire page load
        time.sleep(1)

        # Right now we'll get the elements inside of the page to extract the infos about the Kita

        # Get the Kita address
        try:
            for elem in driver.find_elements_by_xpath('.//a[@title = "Externer Link Wegbeschreibung in Google Maps (Öffnet neuen Tab)"]'):
                kita_address = elem.text
        except:
            kita_address = 'N/A'

        # Get the Kita phone
        try:    
            for elem in driver.find_elements_by_xpath('.//a[@title = "Die Kita anrufen"]'):
                kita_telephone = elem.text
        except:
            kita_telephone = 'N/A'

        # Get the Kita e-mail
        try:    
            for elem in driver.find_elements_by_xpath('.//a[@title = "Eine E-Mail an die Kita senden"]'):
                kita_email = elem.text
        except:
            kita_email = 'N/A'

        # Get the Kita aufnahmealhalter
        try:    
            for elem in driver.find_elements_by_xpath('.//span[@class = "d-block"]'): 
                aufnahmealhalter.append(str(elem.text).split(' '))
                kita_aufnahmealter = str(elem.text).split(' ')
        except:
            kita_aufnahmealter = 'N/A'            

        # Using the Kita aufnahmealhalter we will extract more information
        try:
            kita_distance = ' '.join(aufnahmealhalter[0])
        except:
            kita_distance = 'N/A'        

        try:
            kita_aufnahmealter_jahre = ' '.join(aufnahmealhalter[1])
        except:
            kita_aufnahmealter_jahre = 'N/A'        

        try:
            kita_maximales_platzangebot = ' '.join(aufnahmealhalter[2])
        except:
            kita_maximales_platzangebot = 'N/A'        

        try:
            kita_maximales_platzangebot_verfugbar = ' '.join(aufnahmealhalter[3])
        except:
            kita_maximales_platzangebot_verfugbar = 'N/A'        

        # Get the Kita website
        try:
            for elem in driver.find_elements_by_xpath('.//a[@title = "Externer Link Homepage der Kita (Öffnet neuen Tab)"]'):
                kita_website = elem.text
        except:
            kita_website = 'N/A'            



        # Store everthing in an array using a tuple
        kita_info.append((kita_address,
                          kita_website,
                          kita_telephone,
                          kita_email,
                          kita_aufnahmealter,
                          kita_distance,
                          kita_aufnahmealter_jahre,
                          kita_maximales_platzangebot,
                          kita_maximales_platzangebot_verfugbar                  
                         ))

        # Transform in a pandas dataframe just for filtering
        df = pd.DataFrame(kita_info)

        # Put the right columns
        df.columns = ['address',
                      'website',
                      'phone',
                      'email',
                      'aufnahmealter',
                      'distance',
                      'aufnahmealter_jahre',
                      'maximales_platzangebot',
                      'maximales_platzangebot_verfugbar' 
                     ]
        
        # Close the window
        driver.close()
        
    except:
        print('Invalid kita listing')
        
    return df

In [11]:
# Test
url_kita = \
    'https://kita-navigator.berlin.de/einrichtungen/363?input=Kolberger%20Str.%2C%2013357%20Berlin%2C%20Alemanha&betb=2-2020&einfacheSuche=false&entfernung=5&lat=52.5455402&lon=13.3773108&seite=1&index=1'

df = get_kita_infos(url_kita)
df

,address,website,phone,email,aufnahmealter,distance,aufnahmealter_jahre,maximales_platzangebot,maximales_platzangebot_verfugbar
0,"Wiesenstr. 25, 13357 Berlin",http://www.unionhilfswerk.de,+49-30-4622076,kita.wiesenstrasse@unionhilfswerk.de,"[davon, unter, 3, Jahre:, 30]",0.45 km,3 Monate,87,davon unter 3 Jahre: 30


In [12]:
# Load list of Kitas
df_kitas_url = pd.read_csv('df_kitas_url.csv')
df_kitas_url.head(5)

,kita_url
0,https://www.kita-navigator.berlin.de/einrichtu...
1,https://www.kita-navigator.berlin.de/einrichtu...
2,https://www.kita-navigator.berlin.de/einrichtu...
3,https://www.kita-navigator.berlin.de/einrichtu...
4,https://www.kita-navigator.berlin.de/einrichtu...


In [13]:
# Empty pandas dataframe for store all infos
df_kitas_information = pd.DataFrame()

for index, row in df_kitas_url.iterrows():
    
    # Get th information from the URL using our function and store in a pandas dataframe
    try:
        df = get_kita_infos(row.kita_url)
    
        # Salve the record inside our dataframe that we will use for analysis
        df_kitas_information = df_kitas_information.append(df)
        
    except:
        print(f'Invalid Kita: {row.kita_url}')

Invalid kita listing
Invalid Kita: https://www.kita-navigator.berlin.de/einrichtungen/742?input=Arnold-Knoblauch-Ring%2C%2014109%20Berlin%2C%20Alemanha&betb=2-2020&einfacheSuche=true&entfernung=5&lat=52.4099966&lon=13.137335&seite=2&index=14
Invalid kita listing
Invalid Kita: https://www.kita-navigator.berlin.de/einrichtungen/568?input=Arnold-Knoblauch-Ring%2C%2014109%20Berlin%2C%20Alemanha&betb=2-2020&einfacheSuche=true&entfernung=5&lat=52.4099966&lon=13.137335&seite=1&index=0
Invalid kita listing
Invalid Kita: https://www.kita-navigator.berlin.de/einrichtungen/1823?input=Arnold-Knoblauch-Ring%2C%2014109%20Berlin%2C%20Alemanha&betb=2-2020&einfacheSuche=true&entfernung=5&lat=52.4099966&lon=13.137335&seite=1&index=2
Invalid kita listing
Invalid Kita: https://www.kita-navigator.berlin.de/einrichtungen/1088?input=Arnold-Knoblauch-Ring%2C%2014109%20Berlin%2C%20Alemanha&betb=2-2020&einfacheSuche=true&entfernung=5&lat=52.4099966&lon=13.137335&seite=2&index=13
Invalid kita listing
Invalid Kit

In [14]:
df_kitas_information

,address,website,phone,email,aufnahmealter,distance,aufnahmealter_jahre,maximales_platzangebot,maximales_platzangebot_verfugbar
0,"Zum Heckeshorn 33, 14109 Berlin",http://www.berliner-waldkindergarten.de,+49-176-81751568,info@berliner-waldkindergarten.de,"[davon, unter, 3, Jahre:, 3]",2.8 km,24 Monate,21,davon unter 3 Jahre: 3
0,"Sakrower Landstr. 56, 14089 Berlin",http://www.kita-mariaehimmelfahrt.cfj-caritas-...,+49-30-3656021,kita-mariaehimmelfahrt@t-online.de,"[davon, unter, 3, Jahre:, 25]",4.63 km,12 Monate,84,davon unter 3 Jahre: 25
0,"Zum Heckeshorn 40, 14109 Berlin",http://www.montessori-friends.de/kinderhaus-am...,+49-30-23326400,kita@montessori-friends.de,"[davon, unter, 3, Jahre:, 10]",2.88 km,24 Monate,30,davon unter 3 Jahre: 10
0,"Königstr. 35C, 14109 Berlin",http://www.jaegerhof-berlin.de,+49-30-80587121,info@jaegerhof-berlin.de,"[davon, unter, 3, Jahre:, 20]",2.53 km,12 Monate,55,davon unter 3 Jahre: 20
0,"Von-der-Trenck-Str. 4, 14129 Berlin",http://www.tandembtl.de,+49-30-80402003,kita-zak@tandembtl.de,[90],4.55 km,12 Monate,90,N/A
0,"Normannenstr. 1, 14129 Berlin",http://www.giardino-dei-bambini.de,+49-30-36752100,kontakt@giardino-dei-bambini.de,"[davon, unter, 3, Jahre:, 1]",4.56 km,30 Monate,20,davon unter 3 Jahre: 1
0,"Gößweinsteiner Gang 21, 14089 Berlin",http://www.kita-traumhaus-kladow.de,+49-30-3657458,info@kita-traumhaus-kladow.de,"[davon, unter, 3, Jahre:, 13]",4.4 km,12 Monate,53,davon unter 3 Jahre: 13


Several Kitas that doesn't contains some info have been leftover. This will be fixed soon.

In [15]:
df_kitas_information.to_csv('df_kitas_information.csv', index=True)